In [10]:
import sqlite3
import io
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('batcallsv14.db')
cursor = conn.cursor()

# Retrieve the image data as a byte array
query = "SELECT arr, target FROM batcalls"


df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()


# Function to convert binary data to NumPy array
def convert_to_array(binary_data):
    return np.frombuffer(binary_data, dtype=np.int16)

# Apply the conversion function to the entire column
spectrogram_arr = df['arr'].apply(convert_to_array)

# Replace the previous column with the extracted features
df = df.drop('arr', axis=1)
df_spectrogram = spectrogram_arr.apply(pd.Series)

df_spectrogram.info()
statistics = df_spectrogram.describe()
print(f'Length of DataFrame: {len(df_spectrogram)}')

statistics_count = statistics.loc['count']
nan_indexes = statistics_count[statistics_count < len(df_spectrogram)]

statistics_count.to_csv('statistics_count.csv')
nan_indexes.to_csv('nan_indexes.csv')
statistics.to_csv('BatHunting_stats.csv', index=False)

# Concatenate the split columns with the original DataFrame
df = pd.concat([df, df_spectrogram], axis=1)

#print(df)









<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51719 entries, 0 to 51718
Columns: 8884 entries, 0 to 8883
dtypes: float64(8884)
memory usage: 3.4 GB
Length of DataFrame: 51719


In [8]:
# view data gathered the provided way
import librosa
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io
import sqlite3
import os
from scipy.signal import butter, lfilter
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Flatten, MaxPooling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.activations import relu, tanh, linear
from tensorflow.keras.utils import Progbar
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
from joblib import Parallel, delayed #Paralleize calculation

# Select specific species for classification
targets = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18] # Großer Abendsegler - Nyctalus noctula, Zwergfledermaus - Pipistrellus pipistrellus

all_rows = []

# Make array from the binary data in the database
def _convert_array(text):
	out = io.BytesIO(text)
	out.seek(0)
	return np.load(out, allow_pickle=True)

# Make sqlite3 binary from the given array (probably for saving the array in the database)
def _adapt_array(arr):
	out = io.BytesIO()
	np.save(out, arr)
	out.seek(0)
	return sqlite3.Binary(out.read())

sqlite3.register_adapter(np.ndarray, _adapt_array)
sqlite3.register_converter("ARRAY", _convert_array)


def view_data(target):
    # Connect to the SQLite database
    conn = sqlite3.connect('batcallsv14.db', detect_types=True)
    cursor = conn.cursor()
    # Retrieve the image data as a byte array
    query = f"SELECT arr FROM batcalls where target = {target}"
    df = pd.read_sql_query(query, conn)
    df = pd.DataFrame(df['arr'].tolist())
    
    statistics = df.describe()
    name = f'statistics/Bat.describe_target{target}.csv'
    statistics.to_csv(name)
    return df

dataframes = Parallel(n_jobs=-3, prefer="threads")(delayed(view_data)(target) for target in targets)
df = pd.concat(dataframes)
statistics = df.describe()
statistics.to_csv('full_statistic.csv')

# Total NaN count across the DataFrame
total_nan_count = df.isna().sum().sum()
# NaN count per column
nan_count_per_column = df.isna().sum()
print("Total NaN count:", total_nan_count)
print("NaN count per column:\n", nan_count_per_column)



Num GPUs Available:  1
Total NaN count: 0
NaN count per column:
 0       0
1       0
2       0
3       0
4       0
       ..
4405    0
4406    0
4407    0
4408    0
4409    0
Length: 4410, dtype: int64


In [4]:
import pandas as pd

# Sample DataFrame with a single column containing arrays
data = {'arrays': [[1, 2, 3], [4, 5, 6], [7, 8, 9]]}
df = pd.DataFrame(data)

# Convert each array from the DataFrame into a separate row in the new DataFrame
new_df = pd.DataFrame(df['arrays'].tolist())

print(new_df)


   0  1  2
0  1  2  3
1  4  5  6
2  7  8  9
